In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import sys
import cv2
import os
from scipy import ndimage as nd
from skimage.filters import roberts, sobel, scharr, prewitt
from sklearn import metrics

In [ ]:
img_path = "/content/drive/MyDrive/train/"
img_path1 = "/content/drive/MyDrive/test/"

X_train =[]
y_train = []
X_test = []
y_test = []

#Next three cells are preparing train and test dataset for multiclass classification

In [ ]:
for image in os.listdir(img_path):  # iterate through each file
    if image.split('.')[0][:3] == 'cra':
      y_train.append(1)
    elif image.split('.')[0][:3] == 'oil':
      y_train.append(2)
    elif image.split('.')[0][:3] == 'glu':
      y_train.append(3)
    elif image.split('.')[0][:3] == 'gra':
      y_train.append(4)
    elif image.split('.')[0][:3] == 'rou':
      y_train.append(5)
    else:
        y_train.append(0)
    input_img = cv2.imread(img_path + image)  # Read images
    img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (105, 105))
    X_train.append(img)

In [ ]:
for image in os.listdir(img_path1):  # iterate through each file

    if image.split('.')[0][:3] == 'cra':
      y_test.append(1)
    elif image.split('.')[0][:3] == 'oil':
      y_test.append(2)
    elif image.split('.')[0][:3] == 'glu':
      y_test.append(3)
    elif image.split('.')[0][:3] == 'gra':
      y_test.append(4)
    elif image.split('.')[0][:3] == 'rou':
      y_test.append(5)
    else:
        y_test.append(0)

    input_img = cv2.imread(img_path1 + image)  # Read images
    img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (105, 105))
    X_test.append(img)

In [ ]:
X_train= np.array(X_train)
y_train= np.array(y_train)
X_test= np.array(X_test)
y_test= np.array(y_test)

#Next four cells are preparing train and test dataset for binary classification

In [ ]:
X_train1 =[]
y_train1 = []
X_test1 = []
y_test1 = []

In [ ]:
for image in os.listdir(img_path):  # iterate through each file
    if image.split('.')[0][:3] in ['cra','oil','glu','gra','rou']:
      y_train1.append(1)
    else:
      y_train1.append(0)
    input_img = cv2.imread(img_path + image)  # Read images
    img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (105, 105))
    X_train1.append(img)

In [ ]:
for image in os.listdir(img_path1):  # iterate through each file
    if image.split('.')[0][:3] in ['cra','oil','glu','gra','rou']:
      y_test1.append(1)
    else:
      y_test1.append(0)
    input_img = cv2.imread(img_path1 + image)  # Read images
    img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (105, 105))
    X_test1.append(img)

In [ ]:
X_train1= np.array(X_train1)
y_train1= np.array(y_train1)
X_test1= np.array(X_test1)
y_test1= np.array(y_test1)

In [ ]:
print(len(y_test1
          ))

190


In [ ]:
# input shape is (n, x, y, c) - number of images, x, y, and channels
def feature_extractor(dataset):
    x_train = dataset
    image_dataset = pd.DataFrame()
    for image in range(x_train.shape[0]):  #iterate through each file
        #print(image)

        df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
        #Reset dataframe to blank after each loop.

        input_img = x_train[image, :,:]
        img = input_img
    ################################################################
    #START ADDING DATA TO THE DATAFRAME
    #Add feature extractors, e.g. edge detection, smoothing, etc.

         # FEATURE 1 - Pixel values

        #Add pixel values to the data frame
        pixel_values = img.reshape(-1)/255.0
        df['Pixel_Value'] = pixel_values  #Pixel value itself as a feature
        #df['Image_Name'] = image   #Capture image name as we read multiple images

        # ROBERTS EDGE
        edge_roberts = roberts(img)
        edge_roberts1 = edge_roberts.reshape(-1)
        df['Roberts'] = edge_roberts1

        # SOBEL
        edge_sobel = sobel(img)
        edge_sobel1 = edge_sobel.reshape(-1)
        df['Sobel'] = edge_sobel1

        # SCHARR
        edge_scharr = scharr(img)
        edge_scharr1 = edge_scharr.reshape(-1)
        df['Scharr'] = edge_scharr1

        # PREWITT
        edge_prewitt = prewitt(img)
        edge_prewitt1 = edge_prewitt.reshape(-1)
        df['Prewitt'] = edge_prewitt1

        # CANNY EDGE
        img1 = np.uint8(img)
        edges = cv2.Canny(img1, 100, 200)  # Image, min and max values
        edges1 = edges.reshape(-1)/255.0
        df['Canny_Edge'] = edges1

        # GAUSSIAN with sigma=3
        gaussian_img = nd.gaussian_filter(img, sigma=3)
        gaussian_img1 = gaussian_img.reshape(-1)/255.0
        df['Gaussian3'] = gaussian_img1

        median_img = nd.median_filter(img, size=3)
        median_img1 = median_img.reshape(-1)/255.0
        df['Median3'] = median_img1
        #Append features from current image to the dataset
        image_dataset = pd.concat([image_dataset,df])

    return image_dataset

In [ ]:
image_features = feature_extractor(X_train1)

In [ ]:
n_features = image_features.shape[1]
image_features = np.expand_dims(image_features, axis=0)
x_Train1 = np.reshape(image_features, (X_train1.shape[0], -1))  #Reshape to #images, features

In [ ]:
#Extract features from test data and reshape, just like training data
test_features = feature_extractor(X_test1)
test_features = np.expand_dims(test_features, axis=0)
X_Test1 = np.reshape(test_features, (X_test1.shape[0], -1))


In [ ]:
RF_model = RandomForestClassifier()
#Training the model and estimate training time
RF_model.fit(x_Train1, y_train1)

#Predict on test
test_prediction = RF_model.predict(X_Test1)

##Check model metrics on test dataset.
print ("Accuracy RF_model = ", metrics.accuracy_score(y_test1, test_prediction))

Accuracy RF_model =  0.5263157894736842


In [ ]:
def one_samole_features (image):
  img = cv2.imread(image)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (105, 105))
  img = np.array(img)
  df = pd.DataFrame()
  df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
        #Reset dataframe to blank after each loop.

  # FEATURE 1 - Pixel values

  #Add pixel values to the data frame
  pixel_values = img.reshape(-1)/255.0
  df['Pixel_Value'] = pixel_values  #Pixel value itself as a feature
  #df['Image_Name'] = image   #Capture image name as we read multiple images

  # ROBERTS EDGE
  edge_roberts = roberts(img)
  edge_roberts1 = edge_roberts.reshape(-1)
  df['Roberts'] = edge_roberts1

        # SOBEL
  edge_sobel = sobel(img)
  edge_sobel1 = edge_sobel.reshape(-1)
  df['Sobel'] = edge_sobel1

  # SCHARR
  edge_scharr = scharr(img)
  edge_scharr1 = edge_scharr.reshape(-1)
  df['Scharr'] = edge_scharr1

  # PREWITT
  edge_prewitt = prewitt(img)
  edge_prewitt1 = edge_prewitt.reshape(-1)
  df['Prewitt'] = edge_prewitt1

        # CANNY EDGE
  img1 = np.uint8(img)
  edges = cv2.Canny(img1, 100, 200)  # Image, min and max values
  edges1 = edges.reshape(-1)/255.0
  df['Canny_Edge'] = edges1

        # GAUSSIAN with sigma=3
  gaussian_img = nd.gaussian_filter(img, sigma=3)
  gaussian_img1 = gaussian_img.reshape(-1)/255.0
  df['Gaussian3'] = gaussian_img1

  median_img = nd.median_filter(img, size=3)
  median_img1 = median_img.reshape(-1)/255.0
  df['Median3'] = median_img1
        #Append features from current image to the dataset
  df = np.expand_dims(df, axis=0)
  Test = np.reshape(df, (1, -1))

  return Test



In [ ]:
test = one_samole_features ("/content/drive/MyDrive/augmented_tile_test1/oil17.png")
res = RF_model.predict(test)
print(res)

[0]


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,test_prediction))

              precision    recall  f1-score   support

           0       0.53      1.00      0.69       100
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        25
           3       0.00      0.00      0.00        15
           4       0.00      0.00      0.00        15
           5       0.00      0.00      0.00        15

    accuracy                           0.53       190
   macro avg       0.09      0.17      0.11       190
weighted avg       0.28      0.53      0.36       190



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn import svm
SVM_model = svm.SVC(kernel='poly')
#Training the model and estimate training time
SVM_model.fit(x_Train1, y_train1)

#Predict on test
test_prediction = SVM_model.predict(X_Test1)

##Check model metrics on test dataset.
print ("Accuracy SVM_model = ", metrics.accuracy_score(y_test1, test_prediction))

Accuracy SVM_model =  0.5263157894736842


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test1,test_prediction))

              precision    recall  f1-score   support

           0       0.53      1.00      0.69       100
           1       0.00      0.00      0.00        90

    accuracy                           0.53       190
   macro avg       0.26      0.50      0.34       190
weighted avg       0.28      0.53      0.36       190



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
